In [1]:
import pandas as pd
import numpy as np
import duckdb as duckdb
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt

from dotenv import load_dotenv
import os
import sys
load_dotenv()
sys.path.append(os.getenv('MODULE_PATH_data_preprocessing'))

import st00_add_fixture_ao_banco as st00

In [2]:
caminho = os.getenv('MODULE_PATH_DATA') + 'futebol.db'
SoccerPipeline = st00.SoccerPipeline(caminho)

df = SoccerPipeline.query_execute('SELECT * FROM fixture')

In [3]:
condicoes = [
    df['goals_home'] > df['goals_away'],
    df['goals_home'] == df['goals_away'],
    df['goals_home'] < df['goals_away'],
]

resultados = [
    0,1,2
]

df['resultado'] = np.select(condicoes, resultados, default= 99)

df = df[df['resultado'] != 99]

# Dicionário de mapeamento
country_translation = {
    'England': 'Inglaterra',
    'Spain': 'Espanha',
    'France': 'França',
    'Brazil': 'Brasil',
    'Germany': 'Alemanha',
    'Italy': 'Itália',
    'Netherlands': 'Holanda',
    'Argentina': 'Argentina',
    'Portugal': 'Portugal',
    'Belgium': 'Bélgica'
}

# Substituir os nomes dos países
df['league_country'] = df['league_country'].replace(country_translation)

# Converter a coluna 'fixture_date' para formato datetime
df['fixture_date'] = pd.to_datetime(df['fixture_date'])

# Extrair apenas a parte da data no formato YYYY-MM-DD
df['fixture_date'] = df['fixture_date'].dt.strftime('%Y-%m-%d')

df.sort_values(by='fixture_date', inplace=True)

In [4]:
# Ordenar o DataFrame pela coluna 'fixture_date' dentro de cada 'league_country'
df_sorted = df.sort_values(by=['league_country', 'fixture_date'])

# Calcular os percentis de cada grupo 'league_country'
df_sorted['percentiles'] = df_sorted.groupby('league_country').cumcount() / df_sorted.groupby('league_country')['fixture_date'].transform('count')

# Atribuir os rótulos 'T', 'V' e 'Te' com base nos percentis calculados
df_sorted['split'] = 'T'
df_sorted.loc[df_sorted['percentiles'] >= 0.7, 'split'] = 'T'
df_sorted.loc[df_sorted['percentiles'] >= 0.9, 'split'] = 'X'

In [5]:
# conferindo valores
df_sorted.groupby(['league_country', 'split']).agg(min = ('fixture_date', 'min'),
                                                   max = ('fixture_date', 'max')).reset_index()

,league_country,split,min,max
0,Alemanha,T,2013-08-09,2023-05-06
1,Alemanha,X,2023-05-06,2024-05-18
2,Argentina,T,2015-02-13,2023-02-11
3,Argentina,X,2023-02-11,2023-07-30
4,Brasil,T,2013-05-25,2022-11-02
5,Brasil,X,2022-11-02,2023-12-07
6,Bélgica,T,2013-07-26,2023-05-06
7,Bélgica,X,2023-05-07,2024-05-20
8,Espanha,T,2013-08-17,2023-05-13
9,Espanha,X,2023-05-13,2024-05-19


In [6]:
# separando treino teste

df_treino = df_sorted[df_sorted['split'] == 'T']
df_teste = df_sorted[df_sorted['split'] == 'X']

In [7]:
df_percent = df_treino.groupby('league_country')['resultado'].value_counts(normalize=True).unstack().fillna(0) * 100
df_percent = df_percent.round(2).astype(str) + '%'
df_percent


resultado,0,1,2
league_country,,,
Alemanha,45.25%,24.38%,30.37%
Argentina,42.49%,28.96%,28.55%
Brasil,48.77%,26.89%,24.34%
Bélgica,45.49%,23.59%,30.92%
Espanha,45.72%,25.45%,28.83%
França,43.97%,26.3%,29.73%
Holanda,45.92%,23.41%,30.67%
Inglaterra,45.0%,23.18%,31.82%
Itália,43.25%,25.15%,31.6%


In [8]:
df_percent = df_treino['resultado'].value_counts(normalize=True).fillna(0) * 100
df_percent = df_percent.round(2).reset_index().astype(str) + '%'
df_percent.columns = ['resultado', 'percentual']
df_percent

,resultado,percentual
0,0%,45.08%
1,2%,29.74%
2,1%,25.18%


In [9]:
df_percent = df_teste.groupby('league_country')['resultado'].value_counts(normalize=True).unstack().fillna(0) * 100
df_percent = df_percent.round(2).astype(str) + '%'
df_percent

resultado,0,1,2
league_country,,,
Alemanha,45.1%,26.11%,28.78%
Argentina,45.27%,32.66%,22.06%
Brasil,47.22%,25.42%,27.36%
Bélgica,43.83%,26.23%,29.94%
Espanha,45.32%,27.82%,26.86%
França,41.5%,24.75%,33.75%
Holanda,42.73%,24.63%,32.64%
Inglaterra,46.41%,22.01%,31.58%
Itália,42.45%,28.54%,29.02%


In [10]:
df_percent = df_teste['resultado'].value_counts(normalize=True).fillna(0) * 100
df_percent = df_percent.round(2).reset_index().astype(str) + '%'
df_percent.columns = ['resultado', 'percentual']
df_percent

,resultado,percentual
0,0%,44.49%
1,2%,29.32%
2,1%,26.19%
